Qui sto cercando di capire il ruolo del gradiente che sembra essere uguale ma di segno opposto per i due modes. Se inizializzo i due modes uguali, vanno per sempre avendo parametri uguali e opposti. Ora provato a inizializzarli a caso, vediamo se e dove convergono

In [2]:
import sys
sys.path.append('..')

import utils, selex_dca, indep_sites
import adabmDCA
import selex_distribution, energy_models, tree, data_loading, training, callback, sampling

import torch
from utils import one_hot
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/home/scrotti/Aptamer2025py/selex_dca.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
experiment_id = "Dop8V030"
# round_ids = ["ARN", "R01", "R01CS", "R02N", "R02F", "R02S", "R02SF"]
round_ids = ["ARN", "R01", "R01CS"]

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
dtype = torch.float32

In [ ]:
sequences = []
for round_id in round_ids:
    s = utils.sequences_from_file(experiment_id, round_id, device)
    sequences.append(s)
    print(f"finished {round_id}")

In [194]:
sequences_oh = [one_hot(seq).to(dtype=dtype, device=device) for seq in sequences]

In [18]:
# del sequences

In [60]:
total_reads = torch.Tensor([s.shape[0] for s in sequences_oh]).to(device)
fi0, _, _ = utils.frequences_from_sequences_oh(sequences_oh[0])

In [196]:
tr = tree.Tree()
tr.add_node(-1, name = "R01")
tr.add_node(-1, name = "R01CS")
# tr.add_node(0, name = "R02N")
# tr.add_node(0, name = "R02F")
# tr.add_node(0, name = "R02S")
# tr.add_node(0, name = "R02SF")

mode_names = ["target", "bead"]

selected_modes = torch.BoolTensor(
    [[1, 1],[0, 1]]
).to(device)

In [347]:
import importlib
importlib.reload(selex_distribution)
importlib.reload(energy_models)
importlib.reload(sampling)
importlib.reload(training)
importlib.reload(callback)

<module 'callback' from '/home/scrotti/Aptamer2025py/callback.py'>

In [348]:
L, q = sequences_oh[0][0].shape

k = torch.zeros(L, q, dtype=dtype, device=device)
k = torch.log(fi0)
h_target = torch.randn(L, q, dtype=dtype, device=device)
h_bead = torch.randn(L, q, dtype=dtype, device=device)
# h_bead = h_target.detach().clone()

J_ = torch.randn(L*q, L*q, dtype=dtype, device=device) / torch.sqrt(torch.Tensor([L]).to(dtype=dtype, device=device))
J_ = J_ + J_.t()
J_target = J_.reshape(L, q, L, q)
J_ = torch.randn(L*q, L*q, dtype=dtype, device=device) / torch.sqrt(torch.Tensor([L]).to(dtype=dtype, device=device))
J_ = J_ + J_.t()
J_bead = J_.reshape(L, q, L, q)
# J_bead = J_target.detach().clone()

# J_target = torch.zeros(L, q, L, q, dtype=dtype, device=device)
# J_bead = torch.zeros(L, q, L, q, dtype=dtype, device=device)

Ns0 = energy_models.IndepSites(k)
potts_target = energy_models.Potts(J_target, h_target)
potts_bead = energy_models.Potts(J_bead, h_bead)

ps = selex_distribution.MultiModeDistribution(potts_target, potts_bead, normalized=True)
model = selex_distribution.MultiRoundDistribution(Ns0, ps, tr, selected_modes)

In [349]:
batch_size = 10**6
data_loaders = [data_loading.SelexRoundDataLoader(seq_oh, batch_size=batch_size) for seq_oh in sequences_oh]
n_rounds = len(data_loaders) 

In [350]:
n_chains = 10**4

chains = training.init_chains(n_rounds, n_chains, L, q, device, dtype)
log_weights = torch.zeros(n_rounds, n_chains, device=device, dtype=dtype)

In [351]:
callbacks = [callback.ConvergenceMetricsCallback(), callback.PearsonCovarianceCallback()]

In [352]:
n_sweeps = 10
lr = 0.01
target_pearson = 1
max_epochs = 2000

training.train(model, data_loaders, total_reads, chains, n_sweeps, lr, max_epochs, target_pearson, 
               callbacks=callbacks, log_weights=log_weights)

 0.00%[                             ] Epoch: 0/2000 [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
callbacks[0].plot();

In [ ]:
fig, ax = callbacks[1].plot()
ax.axhline(1, color='r', linestyle='--')

In [ ]:
from IPython.display import display, Latex

potts_zerosum = potts_bead.set_zerosum_gauge()

pl, ax = plt.subplots(figsize=(3,3))
F = selex_dca.get_contact_map(potts_zerosum.J.detach())
im = ax.imshow(F)
ax.set_xlabel("i"); ax.set_ylabel("i")
ax.set_title(experiment_id)
plt.colorbar(im)
plt.tight_layout()
# im.set_clim(-0.0015, 0.0030)
display(Latex("$F_{ij}=\\sqrt {\\sum_{ab}(J_{ij}^{ab})^2}$"))

In [ ]:
from IPython.display import display, Latex

potts_zerosum = potts_target.set_zerosum_gauge()

pl, ax = plt.subplots(figsize=(3,3))
F = selex_dca.get_contact_map(potts_zerosum.J.detach())
im = ax.imshow(F)
ax.set_xlabel("i"); ax.set_ylabel("i")
ax.set_title(experiment_id)
plt.colorbar(im)
plt.tight_layout()
# im.set_clim(-0.0015, 0.0030)
display(Latex("$F_{ij}=\\sqrt {\\sum_{ab}(J_{ij}^{ab})^2}$"))

In [ ]:
plt.scatter(potts_bead.J.cpu().detach().numpy(), potts_target.J.cpu().detach().numpy())